In [1]:
from langchain_community.document_loaders import (PyPDFLoader,PyMuPDFLoader,UnstructuredAPIFileIOLoader)


In [2]:
print("Pypdfloader")
try:
    pypdf_loader=PyPDFLoader("./data/pdf/rag.pdf")
    pypdf_docs=pypdf_loader.load()
    print(pypdf_docs)
    print(f"Total number of pages: {len(pypdf_docs)}")
    print(f"First page: {pypdf_docs[0].page_content}")
    print(f"Metadata: {pypdf_docs[0].metadata}")
except Exception as e:
    print(e)

try:
    pymupdf_loader=PyMuPDFLoader("./data/pdf/rag.pdf")
    pymupdf_docs=pymupdf_loader.load()
    print(pymupdf_docs)
    print(f"Total number of pages: {len(pymupdf_docs)}")
    print(f"First page: {pymupdf_docs[0].page_content}")
    print(f"Metadata: {pymupdf_docs[0].metadata}")
except Exception as e:
    print(e)


Pypdfloader
name 'PyPDFLoader' is not defined
name 'PyMuPDFLoader' is not defined


In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from typing import List

class SmartPDFProcessor:
    
    def __init__(self, chunk_size=1000, chunk_overlap=200):
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len,
            is_separator_regex=False,
        )
    def process_pdf(self, pdf_path:str)->List[Document]:
        loader = PyPDFLoader(pdf_path)
        pages = loader.load()
       

        
        processed_chunks=[]
        for page_num,page in enumerate(pages):
            cleaned_text=self._clean_text(page.page_content)

            if len(cleaned_text.strip())<50:
                continue
            chunks=self.text_splitter.create_documents(texts=[cleaned_text],metadatas=[{"page_num":page_num+1,"source":pdf_path}])

            processed_chunks.extend(chunks)
        return processed_chunks
    def _clean_text(self,text:str)->str:
            text=text.strip()
            text=text.replace("\n"," ")
            text=text.replace("\t"," ")
            text=text.replace("\r"," ")
            return text
                
            

        
            

In [2]:
preprocessor=SmartPDFProcessor()
try:
    smart_chunk=preprocessor.process_pdf("./data/pdf/rag.pdf")
    print(f"Total chunks: {len(smart_chunk)}")
    if smart_chunk:
        print("/nSample chunk metadata: ")   
        for key,value in smart_chunk[0].metadata.items():
            print(f"{key}: {value}")
except Exception as e:
    print(f"Error processing PDF: {str(e)}")

Total chunks: 5
/nSample chunk metadata: 
page_num: 1
source: ./data/pdf/rag.pdf
